# Play with fire! (smoke detection)

## 1. Get data

In [ ]:
! kaggle datasets download -d ahemateja19bec1025/wildfiresmokedataset

## 2. Data preparation

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

random.seed(108)

In [108]:
train_df = pd.read_csv('/kaggle/input/wildfiresmokedataset/train/_annotations.csv')
train_bbox = train_df[['class','xmin','xmax','ymin','ymax']]
train_image_size = train_df[['width','height']].to_numpy()
train_file = train_df[['filename']]

In [109]:
test_df = pd.read_csv('/kaggle/input/wildfiresmokedataset/test/_annotations.csv')
test_bbox = test_df[['class','xmin','xmax','ymin','ymax']]
test_image_size = test_df[['width','height']].to_numpy()
test_file = test_df[['filename']]

In [110]:
valid_df = pd.read_csv('/kaggle/input/wildfiresmokedataset/valid/_annotations.csv')
valid_bbox = valid_df[['class','xmin','xmax','ymin','ymax']]
valid_image_size = valid_df[['width','height']].to_numpy()
valid_file = valid_df[['filename']]

In [103]:

os.makedirs('train/')
os.makedirs('test/')
os.makedirs('valid/')


In [112]:
def BndBox2YoloLine(dir_,info_dict,image_size,box, classList=[]):
    

    xmin = box['xmin'].to_numpy()
    xmax = box['xmax'].to_numpy()
    ymin = box['ymin'].to_numpy()
    ymax = box['ymax'].to_numpy()


    xcen = ((xmin + xmax)) / 2 / image_size[:,0]
    ycen = ((ymin + ymax)) / 2 / image_size[:,1]

    w = ((xmax - xmin)) / image_size[:,0]
    h = ((ymax - ymin)) / image_size[:,1]

    # PR387
    boxName = box['class']
    if boxName not in classList:
        classList.append(boxName)

    classIndex = classList.index(boxName)
    
    for i in range(len(xmin)):
        print_buffer = []
        print_buffer.append("0 {:.3f} {:.3f} {:.3f} {:.3f}".format(xcen[i], ycen[i], w[i], h[i]))
    #print_buffer.append("{:.3f} ".format( xcen.tolist()))
    # Name of the file which we have to save 
        save_file_name = os.path.join(info_dict["filename"][i].replace("jpg", "txt"))
    
    # Save the annotation to disk
        print("\n".join(print_buffer), file= open(dir_+'/'+save_file_name, "w"))

In [113]:
BndBox2YoloLine('train',train_file,train_image_size,train_bbox,[])
BndBox2YoloLine('test',test_file,test_image_size,test_bbox,[])
BndBox2YoloLine('valid',valid_file,valid_image_size,valid_bbox,[])

## 3. Training

In [3]:
!python train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt

python: can't open file '/home/studio-lab-user/sagemaker-studiolab-notebooks/train.py': [Errno 2] No such file or directory


## 4. Analyzation

### 4.1 training process

![confusion_matrix](yolov5/runs/train/exp2/results.png)

### 4.2 Confusion matrix

![confusion_matrix](yolov5/runs/train/exp2/confusion_matrix.png)

### 4.3 Confusion matrix

![confusion_matrix](yolov5/runs/train/exp2/PR_curve.png)

### 4.4 Batch 1: Label vs Predition

![confusion_matrix](yolov5/runs/train/exp2/val_batch0_labels.jpg)

![confusion_matrix](yolov5/runs/train/exp2/val_batch0_pred.jpg)

### 4.5 Batch 2 Label vs Prediction

![confusion_matrix](yolov5/runs/train/exp2/val_batch1_labels.jpg)

![confusion_matrix](yolov5/runs/train/exp2/val_batch1_pred.jpg)

### 4.6 Batch 3 Label vs Prediction

![confusion_matrix](yolov5/runs/train/exp2/val_batch2_labels.jpg)

![confusion_matrix](yolov5/runs/train/exp2/val_batch2_pred.jpg)